# Title: The Fragile Light Switch: Demonstrating Inheritance Pitfalls and the Command Pattern

## Scenario: Building a Smart Home Lighting System

We're building a system that allows us to control lights in different rooms of a house.  We want to be able to turn lights on and off in specific rooms and eventually add more complex lighting controls. We initially choose an inheritance-based approach, where `Room` is the base class, and specific room types (Kitchen, Bathroom, etc.) are subclasses.

## Initial Code (Inheritance Approach)

In [1]:
class Light:
    def __init__(self):
        self.switched_on = False

    def is_switched_on(self):
        return self.switched_on

    def set_switched_on(self, switched_on):
        self.switched_on = switched_on
        print(f"Light switched {'on' if switched_on else 'off'}")

In [2]:
class Room:
    def __init__(self):
        self.light = Light()
        self.name = "Generic Room"

    def switch_lights(self):
        self.light.set_switched_on(not self.light.is_switched_on())
        print(f"Lights switched in {self.name}")

In [3]:
class Kitchen(Room):
    def __init__(self):
        super().__init__()
        self.name = "Kitchen"
        self.oven = "Oven"

    def use_oven(self):
        print(f"Using oven in {self.name}")

class Bathroom(Room):
    def __init__(self):
        super().__init__()
        self.name = "Bathroom"
        self.hot_water = "Hot Water"

    def use_shower(self):
        print(f"Using shower in {self.name}")

class LivingRoom(Room):
    def __init__(self):
        super().__init__()
        self.name = "Living Room"
        self.windows = "Windows"

    def watch_tv(self):
        print(f"Watching TV in {self.name}")

class Bedroom(Room):
    def __init__(self):
        super().__init__()
        self.name = "Bedroom"
        self.music = "Music"

    def listen_music(self):
        print(f"Listening to music in {self.name}")

In [4]:
from typing import List


class House:
    def __init__(self):
        self.rooms: List[Room] = []

    def add_room(self, room: Room):
        self.rooms.append(room)

    def switch_lights_in_all_rooms(self):
        for room in self.rooms:
            room.switch_lights()

In [5]:
house = House()

house.add_room(LivingRoom())
house.add_room(Bathroom())
house.add_room(Kitchen())
house.add_room(Bedroom())

house.switch_lights_in_all_rooms()
house.switch_lights_in_all_rooms()

Light switched on
Lights switched in Living Room
Light switched on
Lights switched in Bathroom
Light switched on
Lights switched in Kitchen
Light switched on
Lights switched in Bedroom
Light switched off
Lights switched in Living Room
Light switched off
Lights switched in Bathroom
Light switched off
Lights switched in Kitchen
Light switched off
Lights switched in Bedroom



## The Fragile Base Class: Demonstrating the Problem

* Let's say we decide to add a logging feature to the `Room` class.
* We want to log the number of times the lights have been switched in a room.
## We modify the base `Room` class like this:

In [ ]:
class Room:  # Redefining Room class to demonstrate the problem
    def __init__(self):
        self.light = Light()
        self.name = "Generic Room"
        self.switch_count = 0
    def switch_lights(self):
        self.light.set_switched_on(not self.light.is_switched_on())
        self.switch_count += 1
        print(f"Lights switched in {self.name} (count: {self.switch_count})")

In [ ]:
house = House()

house.add_room(LivingRoom())
house.add_room(Bathroom())
house.add_room(Kitchen())
house.add_room(Bedroom())

house.switch_lights_in_all_rooms()

Light switched on
Lights switched in Living Room
Light switched on
Lights switched in Bathroom
Light switched on
Lights switched in Kitchen
Light switched on
Lights switched in Bedroom


## The Inheritance Pitfall: Base Class Changes and Broken Subclasses

**Problem:** When you modify a base class (e.g., `Room` to add `switch_count`), subclasses might break if their constructors don't properly initialize the base class attributes.

**Root Cause:** Subclasses need to call `super().__init__()` within their `__init__` methods.  If they don't, they won't initialize the new attributes defined in the redefined base class, leading to errors (like `AttributeError`).

**Solution:**

1.  **Redefine Base Class:** Modify the base class (`Room`) with the new attributes/functionality.
2.  **Update Subclass Constructors:** In each subclass's `__init__` method, **explicitly call `super().__init__()` as the very first line of code**.  This ensures the base class's constructor runs and initializes the inherited attributes.

In [8]:
class Room:  # Redefining Room class to FIX the problem
    def __init__(self):
        self.light = Light()
        self.name = "Generic Room"
        self.switch_count = 0
    def switch_lights(self):
        self.light.set_switched_on(not self.light.is_switched_on())
        self.switch_count += 1
        print(f"Lights switched in {self.name} (count: {self.switch_count})")

In [9]:
class Kitchen(Room):
    def __init__(self):
        super().__init__()
        self.name = "Kitchen"
        self.oven = "Oven"

    def use_oven(self):
        print(f"Using oven in {self.name}")

class Bathroom(Room):
    def __init__(self):
        super().__init__()
        self.name = "Bathroom"
        self.hot_water = "Hot Water"

    def use_shower(self):
        print(f"Using shower in {self.name}")

class LivingRoom(Room):
    def __init__(self):
        super().__init__()
        self.name = "Living Room"
        self.windows = "Windows"

    def watch_tv(self):
        print(f"Watching TV in {self.name}")

class Bedroom(Room):
    def __init__(self):
        super().__init__()
        self.name = "Bedroom"
        self.music = "Music"

    def listen_music(self):
        print(f"Listening to music in {self.name}")

In [11]:
house = House()

house.add_room(LivingRoom())
house.add_room(Bathroom())
house.add_room(Kitchen())
house.add_room(Bedroom())

house.switch_lights_in_all_rooms()
house.switch_lights_in_all_rooms()

Light switched on
Lights switched in Living Room (count: 1)
Light switched on
Lights switched in Bathroom (count: 1)
Light switched on
Lights switched in Kitchen (count: 1)
Light switched on
Lights switched in Bedroom (count: 1)
Light switched off
Lights switched in Living Room (count: 2)
Light switched off
Lights switched in Bathroom (count: 2)
Light switched off
Lights switched in Kitchen (count: 2)
Light switched off
Lights switched in Bedroom (count: 2)


# Scenario 2: Centralized Lighting Control - Whole-House Management

**Background:** A building manager needs a system to control all the lights in an office building, including rooms, hallways, and floor lamps. They need to be able to turn on/off all lights at once (e.g., at the end of the day), control lights by zone (e.g., just the hallway lights), or control lights individually. They also need to be able to schedule lights to turn on/off automatically based on time of day.

# Scattered Logic and Single Responsibility:  Addressing the Source

## Problem Description:

The problem, as the image highlights, is that the *invocation* of the `switchLights` operation (i.e., `light.setSwitchedOn(!light.isSwitchedOn())` or `light.switchLights()`) is happening in multiple classes (e.g., `Room`, `FloorLamp`, and possibly subclasses of `Room`). This violates the Single Responsibility Principle (SRP) because these classes are now responsible not only for their core concerns (being a room, being a floor lamp) but also for *how* the light is switched.

However, a subtler initial problem *before* applying the Command Pattern, the swithcing was done outside the light object.

In [ ]:
class Light:
    def __init__(self):
        self.switched_on = False
    def setSwitchedOn(self, on):
      self.switched_on = on
      print(f"Light set to {'on' if self.switched_on else 'off'}")
    def isSwitchedOn(self):
      return self.switched_on

class Room:
    def __init__(self, light):
        self.light = light

    def switchLights(self):
        self.light.setSwitchedOn(not self.light.isSwitchedOn())

class FloorLamp:
    def __init__(self, light):
        self.light = light

    def switchLights(self):
        self.light.setSwitchedOn(not self.light.isSwitchedOn())

**Explanation of the initial state**

`setSwitchedOn` and `isSwitchedOn` are low-level actions performed on a light
so, for a class to use a light you should use `light.switchLights()` not
`light.setSwitchedOn(not self.light.isSwitchedOn())` because then any class
can toggle lights without worrying how it does this. That would then mean:

`Light` class should be the one controlling the lights, not the Room or the FloorLamp.

In [ ]:
class Light:
    def __init__(self):
        self.switched_on = False

    def switchLights(self):
        self.switched_on = not self.switched_on
        print(f"Light switched {'on' if self.switched_on else 'off'}")

class Room:
    def __init__(self, light):
        self.light = light

    def switchLights(self):
        self.light.switchLights() 

class FloorLamp:
    def __init__(self, light):
        self.light = light

    def switchLights(self):
        self.light.switchLights()

## Persistent SRP Violation: Scattered Invocation, Before the Command Pattern

**Problem Description:**

Even after encapsulating the core light-switching logic *within* the `Light` class (i.e., moving the `switchLights()` method into the `Light` class), the Single Responsibility Principle (SRP) violation *still* persists. While the logic itself is now centralized, the *invocation* of that logic (`light.switchLights()`) is still scattered across multiple classes (`Room`, `FloorLamp`, `LivingRoom`).

**Why is this still a problem?**

1.  **Tight Coupling:** `Room`, `FloorLamp`, and `LivingRoom` are *directly* dependent on the `Light` class's `switchLights()` method. If the `Light` class's API changes (e.g., the `switchLights()` method is renamed or takes arguments), *all* the classes that call it must be modified.
2.  **Limited Flexibility:** If we want to introduce new light-switching behaviors (e.g., dimming, color changing, scheduling), we'd have to modify the `Light` class *and* potentially the classes that call `switchLights()`.
3.  **SRP Violation:** `Room`, `FloorLamp`, and `LivingRoom` have responsibilities *beyond* just being rooms and floor lamps. They're also responsible for *how* the light is switched.

In essence, even though the *logic* is encapsulated, the *control* is still distributed, leading to a design that is less flexible, less maintainable, and more prone to breaking changes. The Command pattern aims to solve this by completely decoupling what needs to be done from *how* it's triggered.

In [ ]:
from abc import ABC, abstractmethod

class Command(ABC):
    @abstractmethod
    def execute(self):
        pass

class SwitchLightsCommand(Command):
    def __init__(self, light):
        self.light: Light = light

    def execute(self):
        self.light.switchLights()

class Light:
    def __init__(self):
        self.switched_on = False

    def switchLights(self):
        self.switched_on = not self.switched_on
        print(f"Light switched {'on' if self.switched_on else 'off'}")

class Room:
    def __init__(self):
        self.command = None

    def setCommand(self, command: Command):
        self.command = command

    def executeCommand(self):
        if self.command:
            self.command.execute()
        else:
            print("No command set for this room.")

class LivingRoom(Room):
  pass

In [ ]:
if __name__ == "__main__":
    # 1. Create a Receiver
    light = Light()

    # 2. Create a Concrete Command, passing in the Receiver
    switch_lights = SwitchLightsCommand(light)

    # 3. Create an Invoker (Room)
    living_room = LivingRoom()

    # 4. Set the Command on the Invoker
    living_room.setCommand(switch_lights)

    # 5. Execute the Command
    living_room.executeCommand() # Light switched on
    living_room.executeCommand() # Light switched off

Light switched on
Light switched off


: 

![Image](./image/example.png)

# Command Pattern: Smart Home Automation

## Scenario

Developing a smart home automation app to control lights in a house. Initial approach involves representing each room as a class inheriting from a base `Room` class with light switching logic.

## Problems with Inheritance Approach

1.  **Fragile Base Class:** Modifying the base `Room` class's light logic risks breaking subclasses (e.g., `Bathroom`, `LivingRoom`).
2.  **Lack of Flexibility:** Difficult to remove features (e.g., light control) from specific rooms (e.g., bathrooms with sensors) without affecting others.
3.  **Code Duplication:** If new elements like a `FloorLamp` need the same light logic, it can't inherit from `Room`, leading to duplicated code.
4.  **SRP Violation:** Classes other than `Light` are controlling the switching which is a problem

## Solution: Command Pattern

1.  **Encapsulate Logic:** Move light-switching logic into the `Light` class itself to ensure consistent behavior across the application.
2.  **Extract Request Details:** Encapsulate the method call (object, method name, arguments) into a separate `Command` class.
3.  **Invoker/Receiver:** Rooms and FloorLamps now hold a `Command` object and have an `execute_command` method. They *invoke* the command but don't *perform* the action directly. The `Light` class becomes the *receiver* of the command, actually executing the light switching.
4. The client has more flexilibity
## Command Pattern Structure

*   **Command Interface:** Declares a single `execute` method.
*   **Concrete Commands:** Implement the `Command` interface (e.g., `SwitchLightsCommand`).
*   **Invoker (Sender):** Initiates the request; holds a reference to a `Command` object (e.g., `Room`).
*   **Receiver:** Contains the business logic to perform the action (e.g., `Light`).
*   **Client:** Creates and configures `Command` objects, passing in request parameters and a receiver instance.

## Benefits of Command Pattern

1.  **Decoupling:** Separates classes that *invoke* operations from classes that *perform* them.
2.  **SRP Adherence:** Each class has a single responsibility.
3.  **Flexibility:** Easily add new commands (e.g., open/close curtains) without modifying existing classes.
4.  **Runtime Configuration:** Decisions are made at runtime by the client, increasing flexibility.
5.  **Queueing/Scheduling:** Commands can be serialized and scheduled, queued, logged, or sent over a network.

## Additional Uses

*   Passing commands as method arguments.
*   Storing commands in other objects.
*   Switching linked commands at runtime.
*   Queueing and scheduling operations.

## Key Takeaway

The Command pattern turns a specific method call into a standalone object, providing flexibility, decoupling, and adherence to the Single Responsibility Principle. It separates what needs to be done from how it's triggered.

![Note](./image/finalnote.png)